In [ ]:
import os
from os.path import join
import sys

# Third-party
import astropy.units as u
import matplotlib.pyplot as pl
from matplotlib import ticker
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline
from scipy.stats import gaussian_kde

import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic, dimensionless

# Custom
import biff

In [ ]:
_path = os.path.abspath("../hdf5lib/")
if _path not in sys.path:
    sys.path.append(_path)
data_path = os.path.abspath("../data/")
import readsnapHDF5 as rsg

---

In [ ]:
import h5py

In [ ]:
with h5py.File("../data/snapdir_255/snap_255.0.hdf5", "r") as f:
    mass_table = f['Header'].attrs['MassTable']
    hubble = f['Header'].attrs['HubbleParam']
    print(f['Header'].attrs['NumPart_Total'])
    
    xyz = f['PartType1']['Coordinates'][:] / hubble * 1000.
    print(xyz.shape)
    
    m_k = mass_table[1] / hubble * np.ones(len(xyz)) # particle mass

In [ ]:
med_xyz = np.median(xyz, axis=0)

# select only particles near center to do KDE
cen_r = 25. # arbitrary threshold
cen_xyz = xyz[np.sum((xyz - med_xyz[None])**2, axis=-1) < cen_r**2]

In [ ]:
kde = gaussian_kde(cen_xyz[::10].T)
density = kde(cen_xyz[::10].T)
center_pos = cen_xyz[::10][density.argmax()]
centered_xyz = xyz - center_pos[None]

In [ ]:
# remove any particles at (0,0,0)
del_ix, = np.where(np.all(centered_xyz == 0., axis=-1))
centered_xyz = np.delete(centered_xyz, del_ix, axis=0)
m_k = np.delete(m_k, del_ix)

In [ ]:
assert len(m_k) == len(centered_xyz)

In [ ]:
# maximum particle radius
r_max = 200.
r = np.sqrt(np.sum(centered_xyz**2, axis=-1))

In [ ]:
fig,axes = pl.subplots(1,3,figsize=(15,5),sharex=True,sharey=True)

style = dict(linestyle='none', marker=',', alpha=0.25)

axes[0].plot(centered_xyz[r<r_max,0], centered_xyz[r<r_max,1], **style)
axes[1].plot(centered_xyz[r<r_max,0], centered_xyz[r<r_max,2], **style)
axes[2].plot(centered_xyz[r<r_max,1], centered_xyz[r<r_max,2], **style)

fig.tight_layout()

axes[0].set_xlim(-r_max,r_max)
axes[0].set_ylim(-r_max,r_max)

In [ ]:
x_xT = np.einsum("ni,nj->nij", centered_xyz[r<r_max], centered_xyz[r<r_max])
I = (m_k[r<r_max,None,None] * x_xT).sum(axis=0)
I

In [ ]:
eigval,eigvec = np.linalg.eig(I)
print("axis ratios:", np.sqrt(eigval[:2]/eigval[2]))

## Rotate halo so coordinates aligned with eigenvectors of inertia tensor

In [ ]:
rot_xyz = np.einsum('ij,ni->nj', eigvec, centered_xyz)

In [ ]:
fig,axes = pl.subplots(1,3,figsize=(15,5),sharex=True,sharey=True)

style = dict(linestyle='none', marker=',', alpha=0.25)

axes[0].plot(rot_xyz[r<r_max,0], rot_xyz[r<r_max,1], **style)
axes[1].plot(rot_xyz[r<r_max,0], rot_xyz[r<r_max,2], **style)
axes[2].plot(rot_xyz[r<r_max,1], rot_xyz[r<r_max,2], **style)

fig.tight_layout()

axes[0].set_xlim(-250,250)
axes[0].set_ylim(-250,250)

## Compute BFE coefficients for the halo

In [ ]:
r_s = 30
S,T = biff.compute_coeffs_discrete(rot_xyz[r<r_max].astype(np.float64), 
                                   mass=m_k[r<r_max].astype(np.float64), 
                                   nmax=16, lmax=16, r_s=r_s)

In [ ]:
from scipy.stats import scoreatpercentile

In [ ]:
n_grid = 64
grid_max = 250.
_grid = np.zeros((3,n_grid*n_grid))
_grid[[0,1]] = np.vstack(map(np.ravel, np.meshgrid(np.linspace(-grid_max,grid_max,n_grid),
                                                   np.linspace(-grid_max,grid_max,n_grid))))

dens = biff.density(np.ascontiguousarray(_grid.T), S, T, M=m_k.sum(), r_s=r_s)

In [ ]:
(dens < 0).sum(), (dens > 0).sum()

In [ ]:
np.log10(dens[dens > 0.])

In [ ]:
scoreatpercentile(np.log10(dens[dens > 0]), [1,99])

In [ ]:
pl.hist(np.log10(dens[dens > 0.]))

In [ ]:
def plot_density_contours(idx, S, T, grid_max=250., n_grid=64):
    _grid = np.zeros((3,n_grid*n_grid))
    _grid[idx] = np.vstack(map(np.ravel, np.meshgrid(np.linspace(-grid_max,grid_max,n_grid),
                                                     np.linspace(-grid_max,grid_max,n_grid))))

    dens = biff.density(np.ascontiguousarray(_grid.T), S, T, M=m_k.sum(), r_s=r_s)
    dens[dens < 0] = np.nan
    
    percs = np.log10([dens[dens > 0].min(), dens[dens > 0].max()])
    levels = np.logspace(percs[0], percs[1], 16)
    
    shp = (n_grid, n_grid)

    fig,ax = pl.subplots(1,1,figsize=(6,6))

#     ax.contour(_grid[idx[0]].reshape(shp), _grid[idx[1]].reshape(shp),
#                dens.reshape(shp), levels=levels,
#                colors='k', linewidths=1)
    ax.contourf(_grid[idx[0]].reshape(shp), _grid[idx[1]].reshape(shp),
                dens.reshape(shp), cmap='Blues', levels=levels,
                locator=ticker.LogLocator())

    ax.set_xlim(-grid_max, grid_max)
    ax.set_ylim(-grid_max, grid_max)
    
    ax.set_xlabel("${}$".format('xyz'[idx[0]]))
    ax.set_ylabel("${}$".format('xyz'[idx[1]]))
    
    tmp = np.array([0,1,2])
    tmp = np.delete(tmp, np.where((tmp==idx[0]) | (tmp == idx[1]))[0])
    ax.set_title("Isodensity contours at ${}=0$".format('xyz'[tmp[0]]))

    fig.tight_layout()
    
    return fig,ax

In [ ]:
fig,ax = plot_density_contours([0,1], S, T, grid_max=r_max)
ax.plot(rot_xyz[r<r_max,0], rot_xyz[r<r_max,1], marker=',', linestyle='none')

In [ ]:
fig,ax = plot_density_contours([0,2], S, T, grid_max=r_max)
ax.plot(rot_xyz[r<r_max,0], rot_xyz[r<r_max,2], marker=',', linestyle='none')

In [ ]:
fig,ax = plot_density_contours([1,2], S, T, grid_max=r_max)
ax.plot(rot_xyz[r<r_max,1], rot_xyz[r<r_max,2], marker=',', linestyle='none')

In [ ]:
pot = biff.SCFPotential(m=m_k.sum(), r_s=r_s, Snlm=S, Tnlm=T)

In [ ]:
w0 = gd.CartesianPhaseSpacePosition(pos=[30.,0,0],
                                    vel=[0.,17.,0.])

In [ ]:
orbit = pot.integrate_orbit(w0, dt=0.1, n_steps=1000) #, Integrator=gi.DOPRI853Integrator)

In [ ]:
_ = orbit.plot(marker='.')